### Introduction to Autograd
https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

In [11]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

# we run the input data through the model through each of its layers to make a prediction.
prediction = model(data) # forward pass

# We use the model’s prediction and the corresponding label to calculate the error (loss). The next step is to backpropagate this error through the network. 
# Backward propagation is kicked off when we call .backward() on the error tensor.
loss = (prediction - labels).sum()
loss.backward() # backward pass

# Next, we load an optimizer, in this case SGD with a learning rate of 0.01 and momentum of 0.9. 
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
optim.step() # iniate gradient descent

print(loss)

tensor(-511.3898, grad_fn=<SumBackward0>)


### Differentiation in Autograd
https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#differentiation-in-autograd

In [13]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

# We create another tensor Q from a and b.
Q = 3*a**3 - b**2

# On fait une descente de gradient
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

# Vérification que les résultats obtenus correspondent bien aux dérivés de la fonction Q
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


### Exclusion from DAG
https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#exclusion-from-the-dag

In [14]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


In [15]:
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

#  we want to finetune the model on a new dataset with 10 labels. 
model.fc = nn.Linear(512, 10)

# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)